## 最新数据的处理
> - 2025年8月31日steam_2025_5k-dataset-games_20250831.json
> - 2025年9月1日
steam_2025_5k-dataset-reviews_20250901.json

### 1. 处理背景
现有steam_.csv仅覆盖至 2019 年，时效性不足，计划下载 2025 年最新 Steam 数据集（如 Steam Games.csv），但新数据多为 JSON 格式（含游戏配置、免费状态等扩展字段），需转换为 CSV 以补充至原有数据集，丰富分析维度。

### 2. 数据预处理
- json转csv
- 初步保留有用特征并分类，删除冗余特征
- 对保留特征逐个分析，进行差异化数据预处理（缺失值处理+异常值处理）

In [2]:
import json
import pandas as pd
import csv

In [3]:
games_json = json.load(open('raw_data/steam_2025_5k-dataset-games_20250831/steam_2025_5k-dataset-games_20250831.json'))
games = games_json['games']
games_json.keys()

dict_keys(['metadata', 'games'])

In [4]:
games = games_json['games']
len(games)

8711

In [5]:
games[0]['app_details']['data'].keys()


dict_keys(['type', 'name', 'steam_appid', 'required_age', 'is_free', 'detailed_description', 'about_the_game', 'short_description', 'fullgame', 'header_image', 'capsule_image', 'capsule_imagev5', 'website', 'pc_requirements', 'mac_requirements', 'linux_requirements', 'developers', 'publishers', 'package_groups', 'platforms', 'categories', 'release_date', 'support_info', 'background', 'background_raw', 'content_descriptors', 'ratings'])

#### 2.1 steam_2025_5k-dataset-reviews数据集字段/特征提取
- 读取评论数据，计算平均游玩时间

In [20]:
# 读取评论数据
review_json = json.load(open('raw_data/steam_2025_5k-dataset-reviews_20250901/steam_2025_5k-dataset-reviews_20250901.json', encoding='utf-8'))
reviews = review_json['reviews']

# 处理评论数据，计算平均游玩时间
game_reviews = {}
for review in reviews:
    appid = review['appid']
    game_reviews[appid] = review['review_data']['query_summary']
    sum_time = 0
    if review['review_data']['reviews']:
        for r in review['review_data']['reviews']:
            sum_time += r["author"]['playtime_forever']
        sum_time = sum_time / len(review['review_data']['reviews'])
    game_reviews[appid]['avg_playtime'] = sum_time

In [21]:
# --- 打印验证部分 ---
print(f"{'AppID':<10} | {'总评论数':<8} | {'好评率':<8} | {'平均游玩时间(分钟)':<12}")
print("-" * 60)

# 取前5个处理好的游戏进行展示
sample_appids = list(game_reviews.keys())[:5]

for appid in sample_appids:
    data = game_reviews[appid]
    
    total_reviews = data.get('total_reviews', 0)
    # 计算好评率 (如果评论数为0则为0)
    pos_reviews = data.get('total_positive', 0)
    pos_rate = f"{(pos_reviews / total_reviews * 100):.1f}%" if total_reviews > 0 else "0%"
    
    avg_time = data.get('avg_playtime', 0)
    
    print(f"{appid:<10} | {total_reviews:<10} | {pos_rate:<10} | {avg_time:<15.2f}")

print("-" * 60)
print(f"总计已处理游戏评论数: {len(game_reviews)}")

AppID      | 总评论数     | 好评率      | 平均游玩时间(分钟)  
------------------------------------------------------------
2210       | 2777       | 87.4%      | 579.61         
2726       | 0          | 0%         | 0.00           
3220       | 93         | 50.5%      | 725.98         
3340       | 111        | 81.1%      | 191.00         
3372       | 0          | 0%         | 0.00           
------------------------------------------------------------
总计已处理游戏评论数: 8711


- 后续数据清洗阶段，注意缺失值处理

#### 2.2 steam_2025_5k-dataset-games数据集字段/特征提取

In [6]:
# data的所有字段
fields = []
num = 0
for game in games:
    if 'data' in game['app_details']:
        fields.extend(game['app_details']['data'].keys())
fields = list(set(fields))
fields
needed_fields = ['metacritic', 'platforms', 'dlc', 'supported_languages',"achievements",
                 'publishers', 'controller_support', 'name', 'categories', 'developers',
                 'steam_appid', 'genres', 'reviews', 'type', 'required_age', 'price_overview',
                 'is_free', 'recommendations', 'ratings', 'release_date',]
# metacritic: 某网站评分 {"score": 80}
# dlc：统计数量代表游戏深度 [2314243,2314234]
# 'supported_languages'： "english,Chinese"
# "achievements": {"total": 24,}
# 'publishers': ["Rename Studio","Perla Oculta"]
# controller_support: 是否支持手柄
# #  "categories": [
#             {
#               "id": 2,
#               "description": "Single-player"
#             },
#             {
#               "id": 28,
#               "description": "Full controller support"
#             },
#             {
#               "id": 62,
#               "description": "Family Sharing"
#             }
#           ],
# "developers": [
# "Black Jacket Studios"
# ],
# "genres": [
# {
#     "id": "1",
#     "description": "Action"
# },
# {
#     "id": "29",
#     "description": "Massively Multiplayer"
# },
# {
#     "id": "37",
#     "description": "Free To Play"
# }
# ],
# "price_overview": {
# "currency": "USD",
# "initial": 899,
# "final": 899,
# "discount_percent": 0,
# "initial_formatted": "",
# "final_formatted": "$8.99"
# },
# "recommendations": {
# "total": 828
# },
# "achievements": {
# "total": 7,
# "highlighted": [
#     {
#     "name": "Complete all steganography puzzles.",
#     "path": "https://cdn.akamai.steamstatic.com/steamcommunity/public/images/apps/746710/decac737d57264b09d18a764e4bf305a4e3297df.jpg"
#     },
# ]
# },
# "ratings": {
# "esrb": {
#     "rating": "t",
#     "descriptors": "Mild Fantasy Violence\r\nMild Lyrics\r\n\r\nMusic Downloads Not Rated by the ESRB"
# },
# "pegi": {
#     "rating": "12",
#     "descriptors": "Bad Language"
# },
# "bbfc": {
#     "rating": "12",
#     "descriptors": "Bad Language"
# },
# "usk": {
#     "rating": "6"
# },
# "oflc": {
#     "rating": "m",
#     "descriptors": "Infrequent sexual references"
# },
# "dejus": {
#     "descriptors": "Free"
# }
# }
# "release_date": {
# "coming_soon": true,
# "date": "To be announced"
# },

a. 按需手动提取与原有数据集匹配 / 扩展的字段
- 基础字段：appid、name、required_age、is_free；
- 扩展字段：游戏配置（PC requirement、Mac requirement）、supported_languages；
- 冗余字段（如 website、域名、游戏宣传图片）暂不提取。

b. 分为4大类：以下是各特征保留的**分析逻辑**（按分类整理）：
- 基本信息
    - `steam_appid`：游戏唯一标识 → 用于关联不同数据集的同一游戏，避免数据混淆（比如匹配旧数据集的游戏评价），是跨数据关联分析的基础。
    - `name`：游戏名称 → 对应实际游戏产品，便于将分析结果落地到具体游戏案例（比如某款热门游戏的特征表现）。
    - `type`：游戏类型（如正式游戏、Demo）→ 区分有效分析样本，Demo无完整数据，筛选出正式游戏才能做核心市场分析。
    - `publishers`：发行商 → 分析发行商的游戏布局偏好（比如某发行商是否集中推某类题材），关联发行资源与游戏热度的关系。
    - `developers`：开发商 → 挖掘开发商的创作风格（比如某开发商是否擅长动作类游戏），分析开发商能力与游戏口碑的关联。
    - `release_date`：发布时间 → 追踪游戏行业的时间趋势（比如近年多人游戏的发布量变化），关联发布节点与市场竞争的关系（比如假期发布的游戏是否更易爆火）。
- 游戏支持
    - `platforms`：设备支持情况 → 设备覆盖范围决定潜在用户规模（仅支持PC的游戏触达用户少于多平台游戏），影响游戏的市场渗透能力。
    - `supported_languages`：语言支持情况 → 语言种类越多，越能覆盖不同区域用户（比如支持中文的游戏可触达国内市场），直接关联区域销量差异。
    - `controller_support`：手柄支持 → 适配手柄的游戏更契合主机/掌机用户的操作习惯，影响特定设备用户的选择意愿（比如主机用户更倾向选支持手柄的游戏）。
- 类别
    - `categories`：功能类别（如多人、单人）→ 功能类型决定游戏的社交属性/玩法定位（多人游戏对应社交需求用户），关联用户群体的需求偏好。
    - `genres`：题材类型（如科幻、武侠）→ 题材对应特定兴趣圈层用户（科幻题材吸引科幻爱好者），分析不同题材的市场受欢迎程度。
- 游戏深度
    - `achievements`：成就系统 → 成就数量/难度反映游戏内容丰富度，成就越多的游戏更易吸引用户长期游玩，关联用户留存时长。
    - `dlc`：扩展内容 → DLC数量反映游戏的长线运营能力，有持续DLC更新的游戏生命周期更长，关联游戏的长期营收潜力。
- 评级
    - `required_age`：年龄限制 → 年龄分级对应目标用户群体（18+游戏瞄准成年用户），影响用户的年龄层分布，关联不同年龄群体的消费偏好。
    - `ratings`：官方评级 → 评级结果反映游戏内容合规性，高评级游戏可能无法在部分地区上架，关联游戏的区域发行范围。
- 评价
    - `metacritic`：媒体评分 → 专业评分反映游戏品质认可度，高分游戏更易获得初期口碑传播，关联游戏的初期销量增速。
    - `reviews`：用户评论 → 用户真实反馈体现游戏的体验短板/亮点，负面评论多的游戏会影响后续用户购买决策。
    - `recommendations`：用户推荐量 → 推荐量越高说明用户满意度越高，高推荐量游戏易形成自发传播，关联游戏的热度扩散速度。
- 价格
    - `price_overview`：价格详情 → 定价策略对应不同消费层级用户（低价游戏覆盖大众用户，高价游戏瞄准核心玩家），直接关联游戏的销量规模。
    - `is_free`：免费标识 → 区分付费/免费商业模式，免费游戏靠内购盈利，付费游戏靠直接购买，关联不同模式下的用户规模与盈利效率差异。

#### 2.3 数据清洗（缺失值、异常值处理）
1. 游戏类别categories&generes：
> steam_2025_5k-dataset-games_20250830.json
- generes共32种，categories共79种
- 痛点 - 一个游戏隶属多个分类（如某游戏同时属于 “Multi-player”“Online Multi-Player”“Valve Anti-Cheat enabled”）
- 解决思路：
    - 采用 Multi-Hot 编码转换：对每个游戏，生成与全局 categories 列表长度一致的 0/1 向量，若游戏属于某类别则对应位置标 1，否则标 0（如某游戏含 “Multi-player” 则向量中该类别位置为 1），既完整保留多分类信息，又转化为结构化数据，适配后续分析（如聚类、推荐系统中计算类别相似度），且操作直观简洁。

> Multi-Hot：apple[0,0,1], orrange[0,1,0],banana[1,0,0]假如一篮子水果，有Apple又有Orange，就编码为[0,1,1]

2. 评论数据（reviews）：
> steam_2025_5k-dataset-reviews_20250901.json
- 痛点：
    - ① 评论数据分散（原始reviews列表按单条评论存储，未按游戏聚合，无法直接关联单款游戏的整体评价）；
    - ② 缺少用户游玩时长的统计维度（仅单用户playtime_forever，无游戏层面的平均游玩时间，难以反映用户粘性）；
    - ③ 部分游戏无评论数据（game_reviews中无对应appid，易导致字段缺失）。

- 解决思路：  
  1. 以`appid`为主关键字，构建`game_reviews`字典”的逻辑，将同一游戏的`query_summary`（含`total_reviews`总评论数、`total_positive`正面评论数、`total_negative`负面评论数、`review_score`评论分数等官方统计指标关联到对应游戏；  
  2. 处理评论数据缺失：对无评论的游戏标记为`None`，避免数据报错，商业模式识别阶段无需用到评论信息，后续再做处理。


3. 时间数据（release_date）：
- 痛点：
    - ① 发布日期格式不统一（原始格式有“Jul 16, 2025”，“15 Nov, 2025”还有乱码格式）；
    - ② 游戏预售，存在“未发布”状态，无实际日期，属于无效分析样本）；
    - ③ 部分日期为空值，且混杂测试版游戏（如名称含“playtest”“demo”“beta”的非正式游戏）。
- 解决思路：  
  - 统一日期格式：适配多种原始格式，并由steamDB手动获取乱码日期（搜索游戏id即可差得），将日期转为“YYYY-MM-DD”标准格式；  
  - 标记未发布状态与处理空值：明确标记未发布游戏，对空日期标为`None`，避免“未发布数据干扰已发布游戏趋势分析”，符合会议中“排除无效数据”的原则；  
  - 过滤非正游戏时间样本：剔除测试版、演示版游戏（数量不多，影响不大），确保仅保留正式游戏的发布日期，避免非核心样本影响时间维度分析结果，呼应会议中“分析样本需为有效正式游戏”的要求。


4. 其他关键字段预处理（结合`needed_fields`与代码逻辑补充）：
- 游戏支持相关（platforms/supported_languages）：  
  - 痛点 - `platforms`为嵌套字典（如`data['platforms']['windows']`）、`supported_languages`为长文本（含多语言逗号分隔，无法直接用于量化分析）。  
  - 解决思路：代码中拆解`platforms`为`platforms_windows`/`platforms_mac`/`platforms_linux`三个布尔字段（True/False反映是否支持），将`supported_languages`转为`support_english`（1=支持英语，0=不支持）的二值字段，既简化分析维度，又契合会议中“设备/语言支持需关联潜在用户规模”的分析逻辑。  

- 价格相关（price_overview/is_free）：  
  - 痛点 - `price_overview`为嵌套字典（含多种货币类型`currency`、多个价格），部分游戏无价格数据（除免费游戏外仍可能缺失）。  
  - 解决思路：
    - 代码中拆解`price_overview`为`price_currency`/`price_initial`/`price_final`/`price_discount_percent`，对无价格数据的免费游戏填充`price_initial`/`price_final`为0（本身数量不多，且上steamDB查询后发现是下架游戏），确保价格字段结构化。 
    - 查看货币种类，发现种类不多，遂直接搜索2025.8.30的汇率转换统一为美元USD 

In [7]:
# 游戏类别
categories = []
genres = []
for game in games:
    if 'data' in game['app_details']:
        data = game['app_details']['data']
        if data['type'] != 'game':
            continue
        if 'categories' in data:
            for category in data['categories']:
                if category['description'] not in categories:
                    categories.append(category['description'])
        if 'genres' in data:
            for genre in data['genres']:
                if genre['description'] not in genres:
                    genres.append(genre['description'])
print(len(categories))
print(len(genres))

79
32


In [8]:
print(len(genres))
for g in genres:
    print(g)

32
Action
RPG
Indie
Casual
Adventure
Strategy
Free To Play
Simulation
Racing
Sports
Early Access
Massively Multiplayer
Utilities
Animation & Modeling
Game Development
Software Training
Design & Illustration
Photo Editing
Violent
Gore
Audio Production
Video Production
Education
Экшены
Инди
Web Publishing
Movie
Sexual Content
Nudity
Acción
Carreras
Deportes


In [9]:
print(len(categories))
for c in categories:
    print(c)

79
Single-player
Full controller support
Family Sharing
Steam Achievements
Stats
Steam Leaderboards
Keyboard Only Option
In-App Purchases
Adjustable Text Size
Custom Volume Controls
Mouse Only Option
Playable without Timed Input
Save Anytime
Steam Cloud
Camera Comfort
Color Alternatives
Adjustable Difficulty
Stereo Sound
Partial Controller Support
Multi-player
PvP
Shared/Split Screen PvP
Co-op
Shared/Split Screen Co-op
Shared/Split Screen
Remote Play Together
Steam Trading Cards
Online PvP
Surround Sound
Cross-Platform Multiplayer
Online Co-op
Captions available
Tracked Controller Support
VR Only
Steam Workshop
LAN Co-op
VR Support
Remote Play on TV
VR Supported
Includes level editor
MMO
Subtitle Options
Touch Only Option
Commentary available
Valve Anti-Cheat enabled
LAN PvP
Includes Source SDK
Remote Play on Phone
Remote Play on Tablet
HDR available
Narrated Game Menus
Steam Timeline
Steam Turn Notifications
Для одного игрока
Достижения Steam
Полная поддержка контроллеров
Remote Play 

In [10]:
review_json = json.load(open('raw_data/steam_2025_5k-dataset-reviews_20250901/steam_2025_5k-dataset-reviews_20250901.json', encoding='utf-8'))
review_json.keys()

dict_keys(['metadata', 'reviews'])

In [11]:
reviews = review_json['reviews']
len(reviews)

8711

In [12]:
all_keys = []
for review in reviews:
    all_keys.extend(review['review_data']['query_summary'].keys())
print(list(set(all_keys)))

['total_positive', 'review_score_desc', 'total_reviews', 'review_score', 'total_negative', 'num_reviews']


In [13]:
game_reviews = {}
for review in reviews:
    appid = review['appid']
    game_reviews[appid] = review['review_data']['query_summary']
    sum_time = 0
    for r in review['review_data']['reviews']:
        sum_time += r["author"]['playtime_forever']
    if sum_time != 0:
        sum_time = sum_time / len(review['review_data']['reviews'])
    game_reviews[appid]['avg_playtime'] = sum_time

game_reviews

{2210: {'num_reviews': 51,
  'review_score': 8,
  'review_score_desc': 'Very Positive',
  'total_positive': 2428,
  'total_negative': 349,
  'total_reviews': 2777,
  'avg_playtime': 579.6078431372549},
 2726: {'num_reviews': 0,
  'review_score': 0,
  'review_score_desc': 'No user reviews',
  'total_positive': 0,
  'total_negative': 0,
  'total_reviews': 0,
  'avg_playtime': 0},
 3220: {'num_reviews': 93,
  'review_score': 5,
  'review_score_desc': 'Mixed',
  'total_positive': 47,
  'total_negative': 46,
  'total_reviews': 93,
  'avg_playtime': 725.9784946236559},
 3340: {'num_reviews': 1,
  'review_score': 8,
  'review_score_desc': 'Very Positive',
  'total_positive': 90,
  'total_negative': 21,
  'total_reviews': 111,
  'avg_playtime': 191.0},
 3372: {'num_reviews': 0,
  'review_score': 0,
  'review_score_desc': 'No user reviews',
  'total_positive': 0,
  'total_negative': 0,
  'total_reviews': 0,
  'avg_playtime': 0},
 3800: {'num_reviews': 2,
  'review_score': 8,
  'review_score_des

In [14]:
# 统计样本中货币种类和数量
df = pd.read_csv('steam_games.csv')  # Read the CSV file into a DataFrame
price_currency = df['price_currency']  # Extract the 'price_currency' column
price_currency.value_counts()  # Count the occurrences of each currency

price_currency
USD    2968
EUR       9
PHP       4
AUD       4
CAD       2
RUB       2
MXN       1
PLN       1
Name: count, dtype: int64

In [15]:

needed_fields = [
    # 基本信息
    'steam_appid',
    'name',
    'type',
    'publishers',
    'developers',
    'release_date',
    #支持
    'platforms',
    'supported_languages',
    'controller_support',
    #类别
    'categories',
    'genres',
    #游戏深度
    "achievements",
    'dlc',
    # 评级
    'required_age',
    'ratings',
    # 评价
    'metacritic',
    'reviews',
    'recommendations',
    # 价格
    'price_overview',
    'is_free',
    
]
import datetime

# 处理每一项，提取你想要的字段
rows = []
for game in games:
    if 'data' not in game['app_details']:
        continue
    data = game['app_details']['data']
    if data['type'] != 'game':
        continue
    if 'playtest' in game['name_from_applist'].lower() or 'demo' in game['name_from_applist'].lower() or 'beta' in game['name_from_applist'].lower():
        continue
    # print(data['release_date'], data['steam_appid'])
    row = {
        # 基本信息
        'steam_appid': data['steam_appid'],
        'name': game['name_from_applist'],
        'type': data['type'],
        # 'developers': ', '.join(d.get('developers', [])) if d.get('developers') else '',
        # 'publishers': ', '.join(d.get('publishers', [])) if d.get('publishers') else '',
        'developers': ';'.join(data['developers']) if 'developers' in data else None,
        'publishers': ';'.join(data['publishers']) if 'publishers' in data else None,
        # date 格式"Jul 16, 2025" 转为2025-07-16
        'release_date': 'Coming Soon' if data['release_date']['coming_soon'] is True
                        else (pd.to_datetime(data['release_date']['date'], format='mixed').strftime('%Y-%m-%d')) if  data['release_date']['date'] != '' else None,
        #支持
        'platforms_windows': data['platforms']['windows'],
        'platforms_mac': data['platforms']['mac'],
        'platforms_linux': data['platforms']['linux'],
        'support_english': (1 if 'english' in data['supported_languages'].lower() else 0) if 'supported_languages' in data else None,
        'supported_languages': data['supported_languages'] if 'supported_languages' in data else None,
        'controller_support': data['controller_support'] if 'controller_support' in data else None,
        #类别
        'categories': ';'.join([c['description'] for c in data['categories']]) if 'categories' in data else None,
        'genres': ';'.join([g['description'] for g in data['genres']]) if 'genres' in data else None,
        #游戏深度
        "achievements": data['achievements']['total'] if 'achievements' in data else None,
        'dlc': len(data['dlc']) if 'dlc' in data else None,
        # 评级
        'required_age': data['required_age'],
        'ratings': data['ratings'],
        # 评价
        'metacritic': data['metacritic'] if 'metacritic' in data else None,
        'reviews': data['reviews'] if 'reviews' in data else None,
        'recommendations': data['recommendations']['total'] if 'recommendations' in data else None,
        'total_reviews': game_reviews[data['steam_appid']]['total_reviews'] if data['steam_appid'] in game_reviews else None,
        'total_positive_reviews': game_reviews[data['steam_appid']]['total_positive'] if data['steam_appid'] in game_reviews else None,
        'total_negative_reviews': game_reviews[data['steam_appid']]['total_negative'] if data['steam_appid'] in game_reviews else None,
        'review_score': game_reviews[data['steam_appid']]['review_score'] if data['steam_appid'] in game_reviews else None,
        'review_score_desc': game_reviews[data['steam_appid']]['review_score_desc'] if data['steam_appid'] in game_reviews else None,
        'avg_playtime': game_reviews[data['steam_appid']]['avg_playtime'] if data['steam_appid'] in game_reviews else None, 
        
        # 价格
        'is_free': data['is_free'] if 'is_free' in data else None,
        'price_currency': data['price_overview']['currency'] if 'price_overview' in data else None,
        'price_initial': data['price_overview']['initial'] if 'price_overview' in data else 0,
        'price_final': data['price_overview']['final'] if 'price_overview' in data else 0,
        'price_discount_percent': data['price_overview']['discount_percent'] if 'price_overview' in data else 0,
    }
    rows.append(row)

# 写入csv文件
with open('steam_games.csv', 'w', encoding='utf-8', newline='') as f:
    writer = csv.DictWriter(f, fieldnames=rows[0].keys())
    writer.writeheader()
    for row in rows:
        writer.writerow(row)

